<a href="https://colab.research.google.com/github/kyo106/CD/blob/master/JPY_COIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import time
from urllib import request, parse
from bs4 import BeautifulSoup as sp
from datetime import datetime
import pytz

# 設定價格
set_price = "0.2106"  # 到價通知
# Line Notify Token
token = 'ELpKx37fme1OFmXex4nbVa20bRJZlrRUamZNpeGb9de'

def send_line_notify(message, token):
    url = "https://notify-api.line.me/api/notify"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"message": message}
    data = parse.urlencode(data).encode()
    req = request.Request(url, data=data, headers=headers)
    with request.urlopen(req) as response:
        res = response.read().decode("utf-8")
    print(f"Line Notify 回應: {res}")

# standard function
def standard(jpy_out, note):
    if jpy_out == set_price:
        send_line_notify(note, token)  # 將 note 發送
    else:
        print(f"未達設定價格: 當前價格 {jpy_out}，設定價格 {set_price}")

# main function
def main():
    tz = pytz.timezone('Asia/Taipei')
    while True:
        current_time = datetime.now(tz)
        current_hour = current_time.hour
        if 9 <= current_hour < 16:  # 只在早上9點到下午4點之間檢查
            local_time = current_time.ctime()
            url = "https://rate.bot.com.tw/xrt?Lang=zh-TW"
            with request.urlopen(url) as response:
                data = response.read().decode("utf-8")
            root = sp(data, "html.parser")
            jpy_in = root.find_all("td")[78].text.replace("買進", "").strip()
            jpy_out = root.find_all("td")[79].text.replace("回售", "").strip()
            s1 = f"\njpy\n銀行買進: {jpy_in}\n銀行賣出: {jpy_out}\nBy Bruno"  # 在銀行賣出價格後加上 "By Bruno"
            note = local_time + s1
            print(f"當前時間: {local_time}")
            print(f"銀行買進價格: {jpy_in}, 銀行賣出價格: {jpy_out}")
            standard(jpy_out, note)
        else:
            print("非檢查時間段內，暫停檢查")
        time.sleep(3600)  # 每小時檢查一次

if __name__ == "__main__":
    main()


非檢查時間段內，暫停檢查


KeyboardInterrupt: 